Game Scenario: Police and Thief Capture


In this version of the game, the player controls a police officer ('P') who starts at the top-left corner of a grid and moves to the bottom-right corner. The objective is to capture all the thieves ('T') along the way. Obstacles ('O') represent buildings or barriers that the police cannot pass through. The police must find the shortest path using the A* algorithm to capture all thieves and reach the bottom-right corner.

In [3]:
import heapq  # Importing heapq to implement the priority queue for A* algorithm
import random  # Importing random to place thieves randomly
import math    # Importing math for distance calculations

# Function to create a grid with police starting at the top-left and thieves randomly placed
def create_grid(size, num_thieves):
    grid = [[' ' for _ in range(size)] for _ in range(size)]  # Create an empty grid filled with spaces
    grid[0][0] = 'P'  # 'P' marks the police starting point at the top-left corner (0,0)
    grid[size-1][size-1] = 'G'  # 'G' marks the goal at the bottom-right corner

    # Place thieves ('T') at random positions, ensuring they are not placed on the police start or goal
    for _ in range(num_thieves):
        thief_x, thief_y = random.randint(0, size-1), random.randint(0, size-1)
        while (thief_x, thief_y) == (0, 0) or (thief_x, thief_y) == (size-1, size-1) or grid[thief_x][thief_y] == 'T':
            thief_x, thief_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[thief_x][thief_y] = 'T'  # Place the thief 'T'

    return grid

# Function to add random obstacles to the grid
def add_obstacles(grid, num_obstacles):
    size = len(grid)
    for _ in range(num_obstacles):
        obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        # Ensure obstacles are not placed on the start 'P', goal 'G', or thieves 'T'
        while grid[obstacle_x][obstacle_y] in ['P', 'G', 'T']:
            obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[obstacle_x][obstacle_y] = 'O'  # Place the obstacle 'O'
    return grid

# Function to check if a position is valid (within bounds and not blocked by an obstacle)
def is_valid_position(grid, x, y):
    size = len(grid)
    return 0 <= x < size and 0 <= y < size and grid[x][y] != 'O'  # Return true if position is valid

# Heuristic function: Calculates the straight-line (Euclidean) distance between the current node and the goal
def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

# A* algorithm function to find the shortest path to capture thieves and reach the goal
def a_star(grid, start, goal):
    size = len(grid)
    # Priority queue to keep track of nodes to explore (using heapq)
    open_list = []
    heapq.heappush(open_list, (0, start))  # Add the start node with priority 0

    # Dictionaries to store the cost to reach each node and the path taken
    g_score = {start: 0}  # Cost from start to current node
    parent = {start: None}  # To reconstruct the path

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Possible directions: up, down, left, right
    thieves_captured = 0
    path = []  # To track the path taken by the police

    while open_list:
        _, current = heapq.heappop(open_list)  # Get the node with the lowest f(n)

        # Add current position to the path
        path.append(current)

        # If we reach the goal, stop the search
        if current == goal:
            break

        # If a thief is captured, increase the count, clear the thief, and print the grid
        if grid[current[0]][current[1]] == 'T':
            thieves_captured += 1
            grid[current[0]][current[1]] = ' '  # Clear the thief after capture
            print(f"\nThief captured at {current}! Updated grid:")
            print_grid_with_path(grid, path)  # Show updated grid with current path

        # Explore neighboring positions (up, down, left, right)
        for direction in directions:
            next_x = current[0] + direction[0]  # Calculate next x-coordinate
            next_y = current[1] + direction[1]  # Calculate next y-coordinate
            next_state = (next_x, next_y)  # Form the next state (position)

            # Check if the next position is valid
            if is_valid_position(grid, next_x, next_y):
                tentative_g_score = g_score[current] + 1  # Cost of moving to the next position

                # If the next state has not been explored or we found a cheaper path to it
                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    g_score[next_state] = tentative_g_score  # Update the cost to reach this state
                    f_score = tentative_g_score + heuristic(next_state, goal)  # Calculate the total cost f(n)
                    heapq.heappush(open_list, (f_score, next_state))  # Add the state to the open list
                    parent[next_state] = current  # Set the current state as the parent of the next state

    # Reconstruct the path from start to goal
    current = goal  # Start from the goal and work backwards
    while current is not None:
        path.append(current)  # Add current position to the path
        current = parent[current]  # Move to the parent of the current state
    path.reverse()  # Reverse the path to start from the beginning (start to goal)

    return path, thieves_captured

# Function to print the grid with lines and the path marked
def print_grid_with_path(grid, path):
    grid_with_path = [row.copy() for row in grid]  # Copy the original grid to preserve it
    for (x, y) in path:
        if grid_with_path[x][y] not in ['P', 'G', 'T']:  # Don't overwrite start 'P', goal 'G', and thieves 'T'
            grid_with_path[x][y] = '*'  # Mark the path with '*'

    # Print the grid with lines to separate cells clearly
    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))  # Print top border line
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')  # Print row with borders between cells
        print('-' * (len(grid_with_path) * 4 + 1))  # Print horizontal line after each row

# Main function to play the game
def police_and_thief():
    # Ask the user for grid size, number of thieves, and number of obstacles
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))
    num_thieves = int(input(f"Enter the number of thieves: "))
    num_obstacles = int(input(f"Enter the number of obstacles (less than {size * size - 2 - num_thieves}): "))  # Make sure obstacles are less than available spaces

    # Create the grid with police, goal, and thieves randomly placed
    grid = create_grid(size, num_thieves)

    # Add random obstacles to the grid
    grid = add_obstacles(grid, num_obstacles)

    # Print the initial grid
    print("\nInitial Grid:")
    print_grid_with_path(grid, [])  # Empty path initially

    # Start the search using A* algorithm
    print("\nPolice capturing thieves and moving to the goal using A* algorithm...\n")
    path, thieves_captured = a_star(grid, (0, 0), (size-1, size-1))  # Perform A* search

    # Print the final grid with the path marked
    print("\nPath to capture all thieves and reach the goal:")
    print_grid_with_path(grid, path)  # Show the path to the goal

    # Print the number of thieves captured
    print(f"\nNumber of thieves captured: {thieves_captured}")

# Run the game
police_and_thief()



Enter the grid size (e.g., 6 for a 6x6 grid): 5
Enter the number of thieves: 4
Enter the number of obstacles (less than 19): 7

Initial Grid:

Grid with Path:
---------------------
| P |   |   | O |   |
---------------------
|   |   |   |   | O |
---------------------
|   |   | O | T | T |
---------------------
| T |   | O |   |   |
---------------------
|   |   | O | T | G |
---------------------

Police capturing thieves and moving to the goal using A* algorithm...


Thief captured at (3, 0)! Updated grid:

Grid with Path:
---------------------
| P | * | * | O |   |
---------------------
| * | * | * |   | O |
---------------------
| * | * | O | T | T |
---------------------
| * |   | O |   |   |
---------------------
|   |   | O | T | G |
---------------------

Thief captured at (2, 3)! Updated grid:

Grid with Path:
---------------------
| P | * | * | O |   |
---------------------
| * | * | * | * | O |
---------------------
| * | * | O | * | T |
---------------------
| * | * | O |  